In [ ]:
import cv2
import numpy as np
import tensorflow as tf

score = 0
game_over = False

left_eye_open = True
right_eye_open = True

model = tf.keras.models.load_model("../OpenClosed/models/opencloseV2.keras")

face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

cap = cv2.VideoCapture(0)

while cap.isOpened():
    
    state, frame = cap.read()

    if state:

        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if not game_over: 
            for (x, y, w, h) in faces:

                roi_gray = gray[y:y + h, x:x + w]
                roi_color = frame[y:y + h, x:x + w]
                eyes = eye_classifier.detectMultiScale(roi_gray, 1.3, 5)

                for (ex, ey, ew, eh) in eyes:

                    left_eye_open = True
                    right_eye_open = True
                    
                    if ey < h // 2:
                        
                        eye_roi = roi_gray[ey:ey + eh, ex:ex + ew]
                        eye_roi = cv2.resize(eye_roi, (50, 50))
                        eye_roi = eye_roi.astype(np.float32) / 255.0
                        eye_roi = eye_roi.reshape(1, 50, 50, 1)

                        pred = model.predict(eye_roi)
                        threshold = 0.5
                        result = (pred > threshold).astype(int)

                        if ex < w // 2:
                            left_eye_open = result
                        elif ex > w//2:
                            right_eye_open = result
            
                if not left_eye_open or not right_eye_open:
                    game_over = True

                else:
                    score+=1
        else:
            cv2.putText(frame, "Press r To Try Again", (30, 80), cv2.FONT_ITALIC, 1, (0, 0, 255), 4)

        cv2.putText(frame, f"Score: {score}", (30, 40), cv2.FONT_ITALIC, 1, (0, 0, 255), 4)
        cv2.imshow("WillYouBlink", frame)
        k = cv2.waitKey(1)
        if k == ord("q"):
            break
        elif k==ord("r") and game_over:
            game_over = False
            score = 0
    else:
        break

cap.release()
cv2.destroyAllWindows()
